# Huggingface Basics
Basic usage of huggingface.

In [9]:
from transformers import pipeline
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

import datasets

## Sentiment analysis on strings

Download a pretrained model and tokenizer for sentiment analysis.

In [2]:
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
Downloading: 100%|██████████| 629/629 [00:00<00:00, 146kB/s]
Downloading: 100%|██████████| 256M/256M [00:09<00:00, 27.9MB/s] 
2022-05-01 08:48:45.338820: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-01 08:48:45.363206: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint a

Use classifier on a single example.

In [3]:
classifier("We are very happy to show you the 🤗 Transformers library.")

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

Use classifiers on a list of examples.

In [4]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


Sentiment classification on a dataset.

TweetEval consists of seven heterogenous tasks in Twitter, all framed as multi-class tweet classification. The tasks include - irony, hate, offensive, stance, emoji, emotion, and sentiment. All tasks have been unified into the same benchmark, with each dataset presented in the same format and with fixed training, validation and test splits.

In [13]:
dataset = datasets.load_dataset("tweet_eval", name='emotion', split="train")

Extracting data files: 100%|██████████| 6/6 [00:00<00:00, 1047.70it/s]
                                                                           

Dataset tweet_eval downloaded and prepared to /Users/Lauren/.cache/huggingface/datasets/tweet_eval/emotion/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


In [16]:
files = dataset["text"]
classifier(files[:4])

[{'label': 'NEGATIVE', 'score': 0.9921145439147949},
 {'label': 'NEGATIVE', 'score': 0.9914141297340393},
 {'label': 'POSITIVE', 'score': 0.9987362027168274},
 {'label': 'NEGATIVE', 'score': 0.6745527982711792}]